# WORK in progress!!!


# GGM omzetten naar MIM

Dit notebook wordt gebruikt om de bestaande versie 1 van [Gemeentelijk Gegevensmodel](https://github.com/Gemeente-Delft/Gemeentelijk-Gegevensmodel) om te zetten naar [MIM (MIM - Metamodel Informatie Modellering)](https://docs.geostandaarden.nl/mim/mim/). 

Hiervoor is het GGM ingelezen in de bijgevoegde Postgres-database en worden op basis van Datamanipulatie de juiste tags en attributen gezet voor: 

1. objecttypes
2. packages
3. enumeraties
4. waardelijsten
5. relaties (associaties en generalisaties)

En vervolgs worden de onderliggende attributen van de juiste tags en attributen voorzien.

Om te kiezen welk onderdeel van het GGM je wil omzetten kies je het GUID (root_guid_import onder 'Configuratie') van het bijbehorende package. Alles wat daaronder zit zet deze app om. Met name voor test en ontwikkeling is het mogelijk gemaakt sledchts delen om te zetten.    

In [2]:
### Importeer bibliotheken en utils

import os
import pandas as pd
import json
from IPython.display import JSON as JSONDisplay
import requests
import database
from re import sub
import ast
import re

import sqlalchemy as sa
from sqlalchemy.orm import sessionmaker
from sqlalchemy import create_engine, inspect
from sqlalchemy.engine import reflection

from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, Sequence
from sqlalchemy.dialects.postgresql import insert
from sqlalchemy import delete
import uuid
from datetime import datetime

def generateEAUUID():
    return '{' +  f'{uuid.uuid4()}'.upper() + '}'

def clone_model(session, model, **kwargs):
    """Clone an arbitrary sqlalchemy model object without its primary key values."""
    # Ensure the model’s data is loaded before copying.
    #model.id

    table = model.__table__
    non_pk_columns = [k for k in table.columns.keys() if k not in table.primary_key.columns.keys()]
    data = {c: getattr(model, c) for c in non_pk_columns}
    data.update(kwargs)

    clone = model.__class__(**data)
    session.add(clone)
    session.commit()
    return clone

## Configuratie

Met de waarden hieronder configureer je de omzetteing. Als voorbeeld is het GUID van Onderwijs actief. Bij de algehele omzetting zal de root_guid gelijk zijn aan de guid_ggm. Guid_ggm is nodig om de juiste tags en attribute te zetten in het hoofdmodel. 

In [3]:
### Configuratie
db_uri = os.getenv("DATABASE_GGM_URL")
db_uri = "sqlite:///root/gemeentelijk gegevensmodel EA16.qea"

root_guid_import = '{3F3163AA-3858-416c-8786-FCB5A48A8E15}' # Root Import from Source-code
root_guid_archeologie = '{0C761AFF-CE80-46d1-921E-41309F5EF068}' # Root Import from Source-code

datum = datetime.today().strftime('%d-%m-%Y')
excel_output = f'./data/Tabellen Archeologie {datum}.xlsx'

## Inlezen Model

Hieronder wordt het model (vanaf 'root_guid_import' en alles daaronder) ingelezen in het dataframe df voor verdere verwerking. Achtereenvolgens worden ingelezen:

1. alle objecten (packages, classes, enumeraties en waardelijsten)
2. alle attributen bij deze objecten
3. alle datatypes waaraan gerelateerd gaat worden

In [4]:
# alle objecten (packages, classes, enumeraties en waardelijsten)

df_obj_import = database.get_df_objectsHierar(db_uri, root_guid=root_guid_import)
df_obj_import['stereotype_object'] = df_obj_import['stereotype']
df_obj_import.head(5)

df_obj_arch = database.get_df_objectsHierar(db_uri, root_guid=root_guid_archeologie)
df_obj_arch['stereotype_object'] = df_obj_arch['stereotype']
df_obj_arch.head(5)
df_obj_import.head(20)

Root Guid {3F3163AA-3858-416c-8786-FCB5A48A8E15} not found returning all
Root Guid {0C761AFF-CE80-46d1-921E-41309F5EF068} not found returning all


,object_id,stereotype,object_type,name,alias,author,version,objectnote,pdata1,ea_guid,parentid,package_id,note,modifieddate,stereotype_object
0,8,None,Class,Contact,None,abrienen,1.0,None,None,{3BF6985C-06AB-4cfc-9A0E-5CC37C224245},0,8,None,2022-12-12 16:28:10,None
1,9,None,Class,Hotel,None,abrienen,1.0,None,None,{5805FA72-C0CE-43c1-A53F-B81166AEFDD4},0,8,None,2022-12-12 16:28:11,None
2,10,None,Class,Hotelbezoek,None,abrienen,1.0,None,None,{669000E9-25D6-4346-B718-516CDB8B88B7},0,8,None,2022-12-12 16:28:11,None
3,11,None,Class,Verkooppunt,None,abrienen,1.0,None,None,{8F05E7BA-E7ED-45d1-9626-C8D7FBDB2F1B},0,8,None,2022-12-12 16:28:11,None
4,12,None,Class,Werkgelegenheid,None,abrienen,1.0,None,None,{EB35F5B8-9289-49cc-8DF4-8BD20EC662A9},0,8,None,2022-12-12 16:28:11,None
5,13,None,Class,Winkelvloeroppervlak,None,abrienen,1.0,None,None,{0EABA880-434F-41c3-A41D-0002222AAC2A},0,8,None,2022-12-12 16:28:11,None
6,17,proxy,Class,Aanwezige Deelnemer,None,Arjen Brienen,1.0,None,None,{F1E55DC7-0F33-40ea-8713-2E1AC3D7EE8D},0,11,None,2022-12-12 16:28:11,proxy
7,18,None,Class,Agendapunt,None,Arjen Brienen,1.0,None,None,{73FB5212-40ED-40dc-B837-36588840445A},0,11,None,2022-12-12 16:28:11,None
8,19,None,Class,Categorie,None,Arjen Brienen,1.0,None,None,{B9B82B25-5D7F-4d2b-84F9-4DF74525EEAD},0,11,None,2022-12-12 16:28:11,None
9,20,None,Class,Collegelid,None,Arjen Brienen,1.0,None,None,{7B9EDDFD-57F7-4ff2-938F-FDFA3B503DA8},0,11,None,2022-12-12 16:28:11,None


In [7]:
# inlezen alle attributen bij deze objecten

df_attr_import = database.get_df(db_uri, "select * from t_attribute")
df_attr_import = df_attr_import.merge(df_obj_import.rename(columns={'name': 'object_name'})[['object_id', 'object_type', 'stereotype_object', 'object_name']], on='object_id', how='inner')
#df_attr_import[df_attr_import.object_type == 'Referentielijst'].head(5)
df_attr_import.head(5)

# inlezen alle attributen bij deze objecten

df_attr_arch = database.get_df(db_uri, "select * from t_attribute")
df_attr_arch = df_attr_arch.merge(df_obj_arch.rename(columns={'name': 'object_name'})[['object_id', 'object_type', 'stereotype_object']], on='object_id', how='inner')
#df_attr_arch[df_attr_arch.object_type == 'Referentielijst'].head(5)
df_attr_import[df_attr_import.ea_guid == '{1179D2B1-6D62-4153-8891-B5F1EDF15F00}']

,object_id,name,scope,stereotype,containment,isstatic,iscollection,isordered,allowduplicates,lowerbound,...,const,style,classifier,Default,type,ea_guid,styleex,object_type,stereotype_object,object_name


## Uitvoeren omzetting

Hieronder vindt de omzetting plaats. Achtereenvolgens worden de volgende stappen gezet:

1. omzetten algemene packages 
2. omzetten alle objecten onder root_guid_import
3. omzetten alle relaties verbonden aan deze objecten
4. omzetten alle attributen van deze objecten

Bij iedere omzetting worden de volgende waarden gezet:
    
1. attributen van het object conform MIM, waaronder altijd het stereotype
2. een xref-record dat de relatie legt tussen het MIM-concept van het stereotype en het zojuist gezette stereotype. in het xref-record wordt steeds de waarde van het mim_text_... opgenomen.
3. de tags/properties die conmform MIM aan een object toegekend moeten worden. Deze staan steeds opgenomen in de 'lst_prop'-lijsten. 

In [248]:
# Omzetten alle attributen van deze objecten
regex = r'Column\(([a-z]+)(?:\(([0-9]+)\))?(?:.*comment="(.*)")?\)'
regex_remove = [
    r",.*nullable=(False|True)",
    r",.*index=(False|True)",
    r",.*default=[a-z0-9\s]+",
    r",.*@observes\(.*\)",
]


def getTypeValues(str):
    for reg in regex_remove:
        str = re.sub(reg, '', str, flags=re.IGNORECASE)
    
    match = re.match(regex, str, re.IGNORECASE)
    if match:
        return match.group(1), match.group(2), match.group(3) 
    else:
        return 'var', None, None

# array(['var', 'String', 'Integer', 'Float', 'Text', 'Boolean', 'Date'],
# Generieke functie om attributen te wijzigen
def changeAttribute(session, ea_guid):
    
    t_attr = session.query(Attribute).filter_by(ea_guid=row.ea_guid).first()
    dtype, lengte, notes = getTypeValues(t_attr.Default)
    #setattr(t_attr, 'Default', '')
    
    if dtype == 'String' and lengte:
        setattr(t_attr, 'type', f'AN{lengte}')
    elif dtype == 'String':
        setattr(t_attr, 'type', f'AN')
    elif dtype == 'Integer':
        setattr(t_attr, 'type', 'int')
    else:
        setattr(t_attr, 'type', dtype.lower())
        
    if lengte:
        setattr(t_attr, 'length', lengte)
    if notes:
        setattr(t_attr, 'notes', notes)


# engine, suppose it has two tables 'user' and 'address' set up
engine = create_engine(db_uri)

# reflect the tables
Base = automap_base()
Base.prepare(autoload_with=engine)
# mapped classes are now created with names by default
# matching that of the table name.
Attribute = Base.classes.t_attribute
Object = Base.classes.t_object
XRef = Base.classes.t_xref
AttributeTag = Base.classes.t_attributetag


# do transformation
session = Session(engine)

for index, row in df_attr_import.iterrows():
    changeAttribute(session, row['ea_guid'])


session.commit()
engine.dispose()

In [249]:
# engine, suppose it has two tables 'user' and 'address' set up
engine = create_engine(db_uri)

# reflect the tables
Base = automap_base()
Base.prepare(autoload_with=engine)
# mapped classes are now created with names by default
# matching that of the table name.
Attribute = Base.classes.t_attribute
Object = Base.classes.t_object
XRef = Base.classes.t_xref
AttributeTag = Base.classes.t_attributetag


# do transformation
session = Session(engine)
df_obj_inner = df_obj_arch.merge(df_obj_import, how='inner', on='name', suffixes=('_arch', '_import'))
for index, row in df_obj_inner.iterrows():
    df_attr_arch_objid = df_attr_arch[df_attr_arch.object_id == row['object_id_arch']]
    df_attr_import_objid = df_attr_import[df_attr_import.object_id == row['object_id_import']]

    df_attr_outer = pd.merge(df_attr_arch_objid,df_attr_import_objid,on='name',how="outer",indicator=True)
    df_attr_right = df_attr_outer[df_attr_outer['_merge']=='right_only']
    df_attr_left = df_attr_outer[df_attr_outer['_merge']=='left_only']
    
    
    print(f'Object: {row["name"]}')
    for i, row_left in df_attr_left.iterrows():
        print(f'Deleting from Old: {row_left["name"]}')
        session.query(Attribute).filter_by(ea_guid=row_left["ea_guid_x"]).delete()
        
    #lst_right = list(['ea_guid'])
    for i, row_right in df_attr_right.iterrows():
        if row_right['type_y'] != 'var':
            print(f'Adding from New: {row_right["name"]}')
            t_attr_import = session.query(Attribute).filter_by(ea_guid=row_right['ea_guid_y']).first()
            t_attr_clone = clone_model(session, t_attr_import, ea_guid=generateEAUUID(), object_id=row['object_id_arch'])
            #print(f'-{t_attr_clone.ea_guid}, {t_attr_clone.object_id}')

  
    
    
'''
for index, row in df_obj_arch.iterrows():
    t_obj_arch = session.query(Object).filter_by(ea_guid=row.ea_guid).first()
    df_attr_arch_obj = df_attr_arch[df_attr_arch.object_id == t_obj_arch.object_id] 
    
    avail = len(df_obj_import[df_obj_import.name == t_obj_arch.name])
    if avail > 0:
        ea_guid_import = df_obj_import[df_obj_import.name == t_obj_arch.name].iloc[0]['ea_guid'] 
        t_obj_import = session.query(Object).filter_by(ea_guid=ea_guid_import).first()
        if t_obj_import:
            df_attr_import_obj = df_attr_import[df_attr_import.object_id == t_obj_import.object_id] 
            
            df=pd.merge(df_attr_arch_obj,df_attr_import_obj,on='name',how="outer",indicator=True)
            lst_right = list(df[df['_merge']=='right_only']['ea_guid'])
            for attr in lst_right:
                t_attr_import = session.query(Attribute).filter_by(ea_guid=attr).first()
                print(f'Adding {t_attr_import.name} to {t_obj_arch.name}')
            
            print(f'Found: {t_obj_import.name}')
'''
df_attr_arch_objid = df_attr_arch[df_attr_arch.object_id == 5642]
df_attr_import_objid = df_attr_import[df_attr_import.object_id == 5608]

df_attr_outer = pd.merge(df_attr_arch_objid,df_attr_import_objid,on='name',how="outer",indicator=True)
df_attr_right = df_attr_outer[df_attr_outer['_merge']=='right_only']
df_attr_left = df_attr_outer[df_attr_outer['_merge']=='left_only']



session.commit()
engine.dispose()
                                                                                  
df_attr_right.head(5)

Object: Doos
Object: Put
Object: Spoor
Object: Stelling
Object: Vlak
Object: Vondst
Object: Vulling
Object: Artefact
Object: Aardewerk
Object: Bouwaardewerk
Object: Dierlijk_Bot
Object: Glas
Object: Hout
Object: Kleipijp
Object: Leer
Object: Menselijk_Bot
Object: Metaal
Object: Project
Object: Munt
Object: Schelp
Object: Steen
Object: Textiel
Object: Partij
Object: ABR
Object: Bruikleen
Object: Velddocument
Object: Foto
Object: Objectfoto
Object: Objecttekening
Object: Opgravingsfoto
Object: Tekening


,object_id_x,name,scope_x,stereotype_x,containment_x,isstatic_x,iscollection_x,isordered_x,allowduplicates_x,lowerbound_x,...,const_y,style_y,classifier_y,Default_y,type_y,ea_guid_y,styleex_y,object_type_y,stereotype_object_y,_merge
44,NaN,__tablename__,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,None,0,'Def_Artefact',var,{7B46AFED-B6EC-46f3-883C-196C5DCFEE93},volatile=0;union=0;,Class,None,right_only
45,NaN,abr_extras,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,None,0,"relationship('ABR', secondary=assoc_artefact_a...",var,{B1A2AAB0-56CE-48e7-9242-B9BAE2CB1BEE},volatile=0;union=0;,Class,None,right_only
46,NaN,abr_materiaal,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,None,0,"relationship(""ABR"", foreign_keys=[abr_materiaa...",var,{3BFBCDAA-B05F-4f04-9BE9-664CD9FB07C6},volatile=0;union=0;,Class,None,right_only
47,NaN,abr_submateriaal,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,None,0,"relationship(""ABR"", foreign_keys=[abr_submater...",var,{6D810C7F-352B-4b38-A093-853487EF6367},volatile=0;union=0;,Class,None,right_only
48,NaN,artefactsoort,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,None,0,"Column(Enum(DiscrArtefactsoortEnum), index=True)",var,{D0786CC5-A8AA-4a75-9191-0D1C92895E8C},volatile=0;union=0;,Class,None,right_only
49,NaN,doos,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,None,0,"relationship('Doos', backref=""artefacten"")",var,{5B2BD351-E7AC-42e7-9FEE-537648232949},volatile=0;union=0;,Class,None,right_only
50,NaN,doosID,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,None,0,"Column(ForeignKey('Def_Doos.primary_key', defe...",var,{A9077099-4367-4987-9BF2-A0813FB03A71},volatile=0;union=0;,Class,None,right_only
51,NaN,primary_key,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,None,0,"Column(Integer, primary_key=True, autoincremen...",var,{A1562DD0-D578-4860-8009-67053EACCCF3},volatile=0;union=0;,Class,None,right_only
52,NaN,project,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,None,0,"relationship('Project', lazy=""joined"", back_po...",var,{F00F0F8D-D41A-4a02-83E2-E27242003B93},volatile=0;union=0;,Class,None,right_only
53,NaN,projectID,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,None,0,"Column(ForeignKey('Def_Project.primary_key', d...",var,{7E4A5A34-A446-4546-BE5E-8958EB1F9F73},volatile=0;union=0;,Class,None,right_only


In [28]:
lst_objIDs = list(df_obj_arch[df_obj_arch.object_type == 'Class']['object_id'])
df_attr = database.get_df(db_uri, "select * from t_attribute")
df_attr = df_attr.merge(df_obj_arch.rename(columns={'name': 'object_name'})[['object_id', 'object_type', 'stereotype_object', 'object_name']]) 

# Create a Pandas Excel writer using XlsxWriter as the engine.
writer = pd.ExcelWriter(excel_output)
for objID in lst_objIDs:
    df = df_attr[df_attr.object_id == objID]
    if len(df) > 0:
        objectname = df.iloc[0]['object_name']
        df[['name', 'type', 'length', 'notes']].to_excel(writer, sheet_name=objectname)
writer.close()
    
#excel_output
#df_obj_arch
lst_objIDs
df_attr.iloc[0]['object_name']

'Monster_Schelp'

## Utils

Diverse scripts voor analyse

In [117]:
str = 'Column(String(1024), comment="decoratie")'
#str = 'Column(String, comment="decoratie")'
#str = 'Column(String)'

match = re.match(r'Column\(([a-z]+)(\(([0-9]+)\))?(..comment="([a-z]+)")\)?\)', str, re.IGNORECASE)
match = re.match(r'Column\(([a-z]+)(?:\(([0-9]+)\))?(?:.*comment="([a-z]*)")?\)', str, re.IGNORECASE)
match.group(3)

'decoratie'

In [73]:
regex = r'Column\(([a-z]+)(?:\(([0-9]+)\))?(?:.*comment="([a-z]*)")?\)'

def setDatatype(attr):
    match = re.match(regex, attr['Default'], re.IGNORECASE)
    if match:
        return match.group(1)
    else:
        return attr['type']

df_attr_import['new_dt'] = df_attr_import.apply(lambda x: setDatatype(x), axis=1)
df_attr_import['new_dt'].unique()

array(['var', 'String', 'Integer', 'Float', 'Text', 'Boolean', 'Date'],
      dtype=object)

In [85]:
regex = r'Column\(([a-z]+)(?:\(([0-9]+)\))?(?:.*comment="(.*)")?\)'
str = 'Column(String(1024), comment="decoratie")'
str = 'Column(String)'
str = 'column(string(1024), comment="c14 datering mogelijk ja of nee")'
str = 'column(string(1024), comment="alleen invullen in uitzonderlijke situatie, restauratie nodig ?")'

def setDatatype(str):
    match = re.match(regex, str, re.IGNORECASE)
    if match:
        return match.group(1), match.group(2), match.group(3) 
    else:
        return str, None, None
    
setDatatype(str)

('string',
 '1024',
 'alleen invullen in uitzonderlijke situatie, restauratie nodig ?')

In [111]:
def clone_model(session, model, **kwargs):
    """Clone an arbitrary sqlalchemy model object without its primary key values."""
    # Ensure the model’s data is loaded before copying.
    #model.id

    table = model.__table__
    non_pk_columns = [k for k in table.columns.keys() if k not in table.primary_key.columns.keys()]
    data = {c: getattr(model, c) for c in non_pk_columns}
    data.update(kwargs)

    clone = model.__class__(**data)
    session.add(clone)
    session.commit()
    return clone


# engine, suppose it has two tables 'user' and 'address' set up
engine = create_engine(db_uri)

# reflect the tables
Base = automap_base()
Base.prepare(autoload_with=engine)
# mapped classes are now created with names by default
# matching that of the table name.
Attribute = Base.classes.t_attribute
Object = Base.classes.t_object
XRef = Base.classes.t_xref
AttributeTag = Base.classes.t_attributetag

ea = '{49E1EE64-5E78-4842-A718-61B03CE10F38}'
# do transformation
session = Session(engine)

t_obj = session.query(Object).filter_by(ea_guid=ea).first()

new_id = engine.execute(Sequence('object_id_seq')) #t_attributetag
t_obj2 = clone_model(session, t_obj, ea_guid=generateEAUUID(), object_id=new_id)

session.commit()
engine.dispose()

In [108]:
generateEAUUID()

'{CACBCF49-FE99-4D2D-B69E-721127630B50}'

In [106]:
x = uuid.uuid4()
f'{x}'.upper()

'91827D46-9FB3-440D-84B1-94745704AA28'

In [151]:
length = len(df_obj_import[df_obj_import.name == 'Artefact'])
if length > 0: 
    ea = df_obj_import[df_obj_import.name == 'Artefact'].iloc[0]['ea_guid']
else:
    ea = None
    
ea

df_obj_import[df_obj_import.name == 'Artefact']

TypeError: NDFrame.first() missing 1 required positional argument: 'offset'